In [2]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ba1lsqeq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ba1lsqeq
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

# import the dataset which consists the caption and the image name for comparison
df = pd.read_excel("")

# dataset location for generated images
generated_images_folder = ""

,Image,Modified Captions
0,001867.jpg,A white car with light scratches on the front ...
1,001970.jpg,A red car with a light scratch on the front bu...
2,001992.jpg,A blue car with moderate scratches on the fron...
3,002047.jpg,A silver car with light scratches on the front...
4,002050.jpg,A silver car with a moderate scratches on the ...
5,002053.jpg,A white car with moderate scratches on the fro...
6,002089.jpg,A white car with light scratch on the front bu...
7,002137.jpg,A brown car with light scratches on the front ...
8,002149.jpg,A silver car with moderate scratches on the re...
9,002197.jpg,A silver car with a moderate scratches on the ...


In [ ]:
import clip
import os
import torch
from PIL import Image

# below code will take the data frame captions, generate clip scores, add them to the df and compute average
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def calculate_clip_score(image_path, text_prompt):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    text = clip.tokenize([text_prompt]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)
        score = torch.cosine_similarity(image_features, text_features).item()
    return score

clip_scores = []

for index, row in df.iterrows():
    image_name = row["Image"]
    caption = row["Modified Captions"]

    image_path = os.path.join(generated_images_folder, image_name)

    score = calculate_clip_score(image_path, caption)
    clip_scores.append(score)

df["CLIP Score"] = clip_scores

print(f"Average CLIP Score: {sum(clip_scores) / len(clip_scores):.4f}")

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 177MiB/s]


Average CLIP Score: 0.3026
